# A*PA evals

This notebook contains the evals for A*PA.
To reproduce figures from the paper, first unzip `results.zip` using `make unzip_results`.
Alternatively, you can rerun all experiments (takes 10-20 hours) using `make run`.

In [ ]:
import numpy as np
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import json
from pathlib import Path

In [ ]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 1000)

# ensures generated PDFs have a deterministic timestam on August 4th 2023.
import os
os.environ["SOURCE_DATE_EPOCH"] = "1691142813"

# Data reading and preparation

In [ ]:
labelsize=10
markersize=4
linewidth = 0.75

def column_display_name(col):
    d = {
        "divergence": "Divergence",
        "runtime": "Runtime per alignment [s]",
        "runtime_capped": "Runtime per alignment [s]",
        "s_per_pair": "Avg. runtime per alignment [s]",
        "s_per_pair_capped": "Avg. runtime per alignment [s]",
        "length": "Sequence length [bp]",
        "band": "Equivalent band",
        "algo_key": "algorithm",
        "algo_pretty": " ",
    }
    if col in d:
        return d[col]
    return col

dataset_pretty = {'ont-ul-500k': 'ONT reads', 'ont-minion-ul-500k': 'ONT reads + genetic variation'}
dataset_order = list(dataset_pretty.keys())

# Line style:
# - slow (no pruning): dotted
# - normal: solid
# - diagonal-transition: dashed
# Colours:
# edlib/wfa ('extern'): blue/purple
# sh/csh/gcsh: orange -> brown -> green gradient
# noprune/normal/dt: 60% -> 70% -> 85% saturation
colors = {'dijkstra': '#786061', 'sh': "#e87146", 'csh': "#8c662a", 'gcsh': "#257d26"}
dashed = (0, (5, 5))
dotted = (0, (1, 4))
dashed_r1 = (0, (2.5, 5))
algorithm_styles = {
    "edlib": ("#DE4AFF", dashed, 'Edlib'),
    "biwfa": ("#625AFF", '-', 'BiWFA'),
    "dijkstra": (colors['dijkstra'], dashed, 'Dijkstra'),
    "gap": ('#402020', dashed, 'A*-Gap'),
    "frequency": ('#802020', dashed, 'A*-Freq'),
    "sh-noprune": (colors['sh'], dotted, 'SH (no prune)'),
    "sh": (colors['sh'], dashed, 'SH'),
    "csh": (colors['csh'], dashed, 'CSH'),
    "gcsh": (colors['gcsh'], dashed, 'GCSH'),
    "dijkstra-dt": (colors['dijkstra'], '-', 'Dijkstra+DT'),
    "sh-dt": ('#e35522', '-', 'SH+DT'),
    "csh-dt": ('#875a12', '-', 'CSH+DT'),
    "gcsh-dt": ('#0f7a10', '-', 'GCSH+DT'),
    'astarpa': ('#0f7a10', '-', 'GCSH+DT'),
    'unknown': ('#000000', '-', 'Unknown')
}
algorithm_order = list(algorithm_styles.keys())

def get_algorithm_key(row):
    name = row['algo_name']
    if name == 'Edlib': return 'edlib'
    if name == 'Wfa':
        if row['job_algo_Wfa_memorymodel'] == 'MemoryUltraLow':
            return 'biwfa'
        else:
            return 'wfa'
    if name == 'AstarPa':
        t = row['job_algo_AstarPa_heuristic_type']
        dt = row['job_algo_AstarPa_diagonaltransition']
        prune = row['job_algo_AstarPa_heuristic_prune'] if 'job_algo_AstarPa_heuristic_prune' in row else 'Both'
        if t == 'None':
            key = 'dijkstra'
        else:
            key = t.lower()
        if t != 'None' and prune == 'None':
            key += '-noprune'
        if dt:
            key += '-dt'
        if key == 'gapcost': return 'unknown'
        return key
    return 'unknown'

# Returns display color, style, and name for an algorithm
def algorithm_display(row, split, default_r=None):
    (c, l, n) = algorithm_styles[row['algo_key']]
    if 'length' in split and 'algo_key' not in split:
        n = f'$n=10^{int(math.log10(row.length))}$'
        if 'r' in split and row.r == 1:
            c = '#000000'
            #l = dashed_r1
    if 'r' in split:
        if 'algo_key' not in split and row.r == 1:
            c = '#000000'
            pass
        if row.r != 0 and (default_r is None or row.r != default_r):
            n += f' ($r={row.r}$)'
    
    return (c, l, n)

In [ ]:
def read_results(path):
    # - Read a json file
    # - Rename json fields from a_b to a-b
    # - Flatten into dataframe
    # - Flatten algorithm params into a few fields:
    #   - algo_name: the type of algorithm
    #   - algo_full: the json-string of algorithm parameters
    # - Rename and compute some common columns:
    #   - error-rate
    #   - length
    #   - s_per_pair
    #   - p_correct
    
    json_path = Path(path)
    data = json.loads(json_path.read_text())
    
    # Remove underscores from all keys
    def remove_underscores(o):
        if isinstance(o, list):
            return [remove_underscores(v) for v in o]
        if isinstance(o, dict):
            return {k.replace('_', ''): remove_underscores(v) for k, v in o.items()}
        return o
    
    data = remove_underscores(data)

    # Clean up algo columns
    for x in data:
        name = list(x['job']['algo'].keys())[0]
        obj = x['job']['algo']
        obj['name'] = name
        x['algo_name'] = name
        x['algo_full'] = json.dumps(obj)
        #del x['job']['algo']
        if 'Ok' in x['output']:
            del x['output']['Ok']['costs']

    # Flatten the js
    df = pd.json_normalize(data, sep='_')
    df['algo_key'] = df.apply(get_algorithm_key, axis=1)
    df['algo_pretty'] = df['algo_key'].map(lambda key: algorithm_styles[key][2])
    
    # Convenience renaming
    df = df.rename({'job_dataset_Generated_length': 'length',
                    'job_dataset_Generated_errorrate': 'errorrate',
                    'job_timelimit': 'timelimit',
                    'output_Ok_pcorrect': 'pcorrect',
                    'output_Ok_measured_runtime': 'runtime',
                    'output_Ok_measured_memory': 'memory',
                    'stats_divergence_mean': 'divergence',
                    'job_algo_AstarPa_diagonaltransition': 'dt',
                    'job_algo_AstarPa_heuristic_prune': 'prune',
                    'job_algo_AstarPa_heuristic_r': 'r',
                    'job_algo_AstarPa_heuristic_k': 'k',
                   }, axis='columns')
    
    # Order rows
    df['algo_ord'] = df['algo_key'].map(lambda key: algorithm_order.index(key))
    if 'k' in df.columns:
        df.sort_values(by='k', inplace=True, kind = 'stable')
    if 'r' in df.columns:
        df.sort_values(by='r', inplace=True, kind = 'stable')
    df.sort_values(by='algo_ord', inplace=True, kind = 'stable')
    if 'length' not in df.columns and 'stats_length_mean' in df.columns:
        df['length'] = df.stats_length_mean
    if 'length' in df.columns:
        df.sort_values(by='length', inplace=True, kind = 'stable')
    if 'errorrate' in df.columns:
        df.sort_values(by='errorrate', inplace=True, kind = 'stable')
    # Order by dataset
    if 'job_dataset_File' in df.columns and df.job_dataset_File.notna().all():
        df['dataset'] = df['job_dataset_File'].map(lambda f: Path(f).parent.name)
        df['dataset_ord'] = df['dataset'].map(lambda key: dataset_order.index(key))
        df.sort_values(by='dataset_ord', inplace=True, kind = 'stable')
    
    # Computed columns
    df['costmodel'] = df.apply(lambda row: (row['job_costs_sub'], row['job_costs_open'], row['job_costs_extend']), axis=1)
    df['s_per_pair'] = df['runtime'] / df['stats_seqpairs']
    df['timelimit_per_pair'] = df['timelimit'] / df['stats_seqpairs']
    if 'length' in df.columns and 'output_Ok_stats_expanded' in df.columns:
        df['band'] = df['output_Ok_stats_expanded'] / (df['stats_seqpairs']* df['length'])

    def bad_k(row):
        # Bad k only makes sense for A*PA with SH-based heuristic.
        if row['algo_name'] != 'AstarPa':
            return False
        
        h = row['job_algo_AstarPa_heuristic_type'].lower()
        if h not in ['sh', 'csh', 'gcsh']:
            return False
        
        # Too small k => too many matches
        if row.k < math.log(row.length, 4) + (2 if row.r == 2 else 0):
            return True
        
        # Too large k => not enough potential
        if row.k > row.r/row.divergence:
            return True
        
        # all is fine
        return False

    if 'length' in df.columns:
        df['bad_k'] = df.apply(bad_k, axis=1)

    def runtime_capped(row):
        if not math.isnan(row['runtime']):
            return row['runtime']
        if row['output_Err'] == 'Timeout':
            return row['timelimit']
        return row['timelimit']*1.1
    df['runtime_capped'] = df.apply(runtime_capped, axis = 1)
    df['s_per_pair_capped'] = df['runtime_capped'] / df['stats_seqpairs']
    
    df['editdistance'] = df['stats_insertions'] + df['stats_deletions'] + df['stats_substitutions']
    
    # Some specific fixes
    df = df.fillna({'r': 0}, downcast='infer')
    
    # Remove unsupported algos
    if 'output_Err' in df.columns:
        df = df[df.output_Err != 'Unsupported']
    
    return df

## The one plotting function

In [ ]:
def plot(df,
         name='',
         file=None,
         x='length',
         y='s_per_pair',
         # Column to use for hue and style.
         # Always change both at the same time!
         hue='algo_key',
         style='r',
         # column to use for marker size
         size=None,
         # Logarithmic axes by default
         xlog=True,
         ylog=True,
         ylim=None,
         xlim=None,
         # alph
         alpha=1.0,
         # Use line instead of scatter plot?
         connect=False,
         # Draw a cone from the given filter and x
         cone=None,
         cone_x=3*10**4,
         fit=False,
         line_labels=False,
         categorical=False,
         ax=None,
         width=None,
         height=None,
         default_r=None,
         bad_k=True,
        ):
    
    if df[y].isna().all():
        print(f"All values of {y} are nan.")
        return
    
    df = df[df[y].notnull()]
    assert not df.empty
    
    # We group data by this set of keys.
    split = [hue, style]
    
    # Remove 'r' from the split if not both r=1 and r=2 are present,
    # to prevent redundant (r=1) in plots.
    if 'r' in split and 'r' in df.columns:
        if not (1 in df.r.values and 2 in df.r.values):
            split.remove('r')
    if 'r' in split and 'r' not in df.columns:
        split.remove('r')
    
    # Group the data into datapoints per line
    groups = df.groupby(split, sort=False)
    
    # Not sure if needed actually.
    sns.reset_defaults()
    sns.set_context(None) # 'paper', 'notebook'
    
    # Set up the figure if not provided.
    if ax is None:
        fig, ax = plt.subplots()
        fig.set_size_inches(width if width else 3, height if height else 2, forward=True)
        hasax = False
    else:
        hasax = True

    
    # Set log scales
    ax.set(xscale='log' if xlog else 'linear', yscale='log' if ylog else 'linear')
    
    # limit number of ticks
    if ylog:
        ax.locator_params(axis='y', numticks=6)
    else:
        ax.locator_params(axis='y', nbins=6)
    
    
    # PLOTTING
    
    if not categorical:
        # Show a scatterplot of points.
        # Each group is plotted separately for more control over its style.
        for k, group in groups:
            first_row = group.iloc[0]
            color, linestyle, grouplabel = algorithm_display(first_row, split, default_r)

            ax.plot(x,
                    y,
                    data=group.sort_values(by=x),
                    color=color,
                    linestyle=linestyle if connect else 'None',
                    marker='' if bad_k else 'o',
                    alpha=alpha,
                    dash_capstyle = 'round',
                    label=grouplabel,
                    zorder=2,
                    markersize=markersize,
                    linewidth=linewidth,
                   )
            
            if bad_k:
                # Draw separate scatter plots with red dots for bad k, and dots in the original color for good k.
                ax.scatter(x,
                           y,
                           data=group[group.bad_k].sort_values(by=x),
                           color='red',
                           marker='o',
                           alpha=alpha,
                           zorder=2,
                           s=markersize**2,
                   )
                ax.scatter(x,
                           y,
                           data=group[~group.bad_k].sort_values(by=x),
                           color=color,
                           marker='o',
                           alpha=alpha,
                           zorder=2,
                           s=markersize**2,
                   )
                    
    if categorical:
        # Map algorithms to their color, unless bad_k is set, in which case those are made red.
        hue_vec = df.apply(lambda row: 'red' if bad_k and row.bad_k else algorithm_display(row, split, default_r)[0], axis=1)
        palette = {algorithm_display(row, split, default_r)[0]: algorithm_display(row, split, default_r)[0] for index, row in df.iterrows()} | {'red': 'red'}
        # Overlay a boxplot and swarmplot on top of each other
        sns.swarmplot(data=df,
                        x=x,
                        y=y,
                        hue=hue_vec,
                        palette=palette,
                        ax=ax,
                        size=3,
                        linewidth=0,
                        edgecolor='gray',
                        zorder=10,
                        dodge=False,
        )
        sns.boxplot(data=df,
                    x=x,
                    y=y,
                    zorder=9,
                    ax=ax,
                    boxprops={'facecolor':'None'},
                    showfliers=False,
                    linewidth=linewidth,
                   )
    
    # TEXT
    
    # Title
    if name:
        ax.set_title(name, y=1.05)
    
    # Remove legend
    ax.legend().remove()
    
    # BACKGROUND
    ax.set_facecolor("#F8F8F8")
    ax.set_axisbelow(True) 
    ax.grid(False)
    ax.grid(True, axis="y", which="major", color="white", alpha=1, zorder=0)
    
    
    # AXES
    
    # Labels
    ax.set_xlabel(column_display_name(x))  # weight='bold',
    ax.set_ylabel(column_display_name(y), rotation=0, ha="left")
    ax.yaxis.set_label_coords(-0.10, 1.00)
    
    # Limits
    x_margin = 1.5
    y_margin = 3
    if xlog:
        #xs = df[df[x] > 0][x]
        ax.set_xlim(df[x].min() / x_margin, df[x].max() * x_margin)

    if ylog:
        ax.set_ylim(df[y].min() / y_margin, df[y].max() * y_margin)
    
    # Start linear scales at 0.
    if not xlog and not categorical and x != 'job_costs_open':
        ax.set(xlim=(0,None))
    if not ylog:
        ax.set(ylim=(0,None))
    if ylim is not None:
        ax.set_ylim(ylim[0], ylim[1])
    if xlim is not None:
        ax.set_xlim(xlim[0], xlim[1])
 
    
    # Show bottom spine, and left spine when xlog=false
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(not xlog and not categorical and (xlim is None or xlim[0] == 0))
    
    # Format % scales.
    if x in ['errorrate', 'divergence']:
        ax.xaxis.set_major_formatter(mtick.PercentFormatter(1.0, decimals=0))
    if x == 'k' and not categorical:
        ax.xaxis.set_major_locator(mtick.MultipleLocator(base=2,offset=1))
    
    # Show major ticks
    ax.tick_params(
        axis="both",
        which="major",
        bottom=True,
        top=False,
        left=True,
        right=False,
    )
    # No minor ticks
    ax.tick_params(
        axis="both",
        which="minor",
        bottom=False,
        top=False,
        left=False,
        right=False,
        labelbottom=False,  # labels along the bottom edge are off
    )
    # Do show minor ticks for small log ranges
    #if ylog:
    #     ax.tick_params(axis="y", which="minor", left=True)
    
    
    # CONE
    # Fills the region between x**1 and x**2
    if cone:
        x0 = cone_x
        x_max = x_margin * df[x].max()
        x_range = (x0, x_max)
        
        y0 = df[cone(df) & (df[x] == cone_x)][y].max()
        y_lin = (y0, y0 * (x_max / x0) ** 1)
        y_quad = (y0, y0 * (x_max / x0) ** 2)
        ax.fill_between(x_range, y_lin, y_quad, color="grey", alpha=0.15, zorder=0.4)
        
    # TIME LIMIT
    if y=='runtime_capped' or (y=='s_per_pair_capped' and x != 'length'):
        timelimit = df.timelimit.iloc[0]
        #assert df[df.runtime.isna()].timelimit.eq(timelimit).all()
        # Draw a red line at the timelimit.
        ax.axhline(y=timelimit, color="red", linestyle="-", alpha=1, linewidth=0.5)
        
        # Modify/add the timelimit ticklabel with TL=
        if False:
            ylabels = [x for x in ax.get_yticklabels()]
            found = False
            for i, l in enumerate(ylabels):
                if l.get_position()[1] == timelimit:
                    ylabels[i] = "TL=" + ylabels[i].get_text()
                    found = True
            if found:
                ax.set_yticklabels(ylabels)
            else:
                yticks = list(ax.get_yticks())
                ylabels = list(ax.get_yticklabels())
                yticks.append(timelimit)
                ylabels.append("TLE")
                ax.set_yticks(yticks)
                try:
                    ax.set_yticklabels(ylabels)
                except ValueError:
                    pass
                finally:
                    pass
            
    # POLY FIT

    def angle(slope):
        x_min, x_max = ax.get_xlim()
        y_min, y_max = ax.get_ylim()
        bbox = ax.get_window_extent()
        x_sz = bbox.width
        y_sz = bbox.height
        x_factor = x_sz / (np.log10(x_max) - np.log10(x_min) if xlog else x_max - x_min)
        y_factor = y_sz / (np.log10(y_max) - np.log10(y_min) if ylog else y_max - y_min) 
        slope = slope * y_factor / x_factor
        return math.atan(slope)*180/math.pi
    
    if fit:
        assert x=='length' and xlog and ylog, "Polynomial fits only work in log-log plots with x=length"
        for k, group in groups:
            first_row = group.iloc[0]
            color, linestyle, grouplabel = algorithm_display(first_row, split, default_r)
            fit_label = grouplabel
            
            filtered = group[group.runtime.notnull()]
            ps = filtered[[x,y]].dropna()
            xmin, xmax = filtered[x].min(), filtered[x].max()
            if len(ps) > 1:
                fit = np.polyfit(np.log(ps[x]), np.log(ps[y]), 1)
                f = lambda x: x**fit[0] * np.exp(fit[1])
                # Extra {{ and }} are for the math-mode superscript
                fit_label = f"{grouplabel} $\sim n^{{{fit[0]:0.2f}}}$"

                ymin, ymax = f(xmin), f(xmax)
                # line from xmin to xmax (use plt.axline for infinite line)
                ax.plot([xmin, xmax], [ymin, ymax], color=color, linestyle=linestyle, alpha=1, dash_capstyle = 'round', label=grouplabel, zorder=2, linewidth=linewidth)
                #print(f'Exponent for {k}: {fit[0]:0.2f}')
            else:
                ymax = float('inf')
                fit = [0]
                

            ax.text(
                xmax,
                min(ymax, ax.get_ylim()[1]),
                fit_label,
                color=color,
                ha="right",
                va="bottom",
                size=labelsize,
                alpha=1,
                rotation=angle(fit[0]),
                rotation_mode='anchor',
            )
    if line_labels:
        # If no legend and no fits are shown, show manual labels instead
        for split_key, group in groups:
            first_row = group.iloc[0]
            color, linestyle, grouplabel = algorithm_display(first_row, split, default_r)
            max_idx = group[x].idxmax()
            label_x = group[x][max_idx]
            label_y = min(group[y][max_idx], ax.get_ylim()[1])
            key = split_key[0] if isinstance(split_key, tuple) else split_key
            
            by_x = group[x].argsort()
            last = group.iloc[by_x.iloc[-1]]
            try:
                before = group.iloc[by_x.iloc[-3]]
                slope = (last[y] - before[y])/(last[x] - before[x])
            except:
                slope = 0
            ax.text(
                label_x,
                label_y,
                grouplabel,
                color=color,
                ha="right",
                va="bottom",
                size=labelsize,
                alpha=1,
                rotation=angle(slope),
                rotation_mode='anchor',
            )

    if not hasax:
        if file:
            plt.savefig(f"plots/{file}.pdf", dpi=300, bbox_inches='tight')
            #plt.savefig(f"plots/{file}.png", dpi=300, bbox_inches='tight')
            
    return ax

# Sanity check: CPU frequency = 2600MHz

In [ ]:
df = read_results("results/cache.json")
df = df.rename({'output_Ok_measured_cpufreqstart': 'freqstart','output_Ok_measured_cpufreqend': 'freqend'}, axis='columns')
for c in ['freqstart', 'freqend']:
    print(df[c].min(), df[c].max())
    assert df[c].min() > 2550
    assert df[c].max() < 2650

# Error rate to divergence

In [ ]:
print('Error rate to divergence mapping')
df = read_results("results/memory.json")
display(df[df.length==1000000].pivot_table(index='errorrate', values = 'divergence'))

# Paper figures and tables
## Scaling with length (Fig 4ab, appendix)

In [ ]:
df = read_results("results/scaling_n.json")
df.loc[df['algo_key'] == 'edlib', 'dt'] = False
df.loc[df['algo_key'] == 'biwfa', 'dt'] = True


cone = lambda df: (df['algo_key'] == 'sh') & (df['r'] == (1 if e <= 0.05 else 2))
for e, g in df.groupby('errorrate'):
    default_r = None
    if e == 0.05:
        g = g[g.algo_key != 'csh']
        g = g[g.algo_key != 'csh-dt']
        # at r=2, only SH
        g = g[(g.r != 2) | (g.algo_key == 'sh')]
        # for GCSH, only with DT
        g = g[g.algo_key != 'gcsh' ]
        default_r = 1
    if e == 0.15:
        g = g[g.algo_key != 'csh']
        g = g[g.algo_key != 'csh-dt']
        # at r=1, only GCSH
        g = g[(g.r != 1) | (g.algo_key == 'gcsh') | (g.algo_key == 'gcsh-dt')]
        default_r = 2
    plot(g, file=f'scaling_n_e{e}', x='length', y='s_per_pair', fit=True, cone=cone, cone_x = 3000, width=4.4, height=3, ylim = (10**-3.9, 10**3.9), default_r=default_r)
plt.show()

assert(df[df.output_Err.notna()].output_Err.apply(lambda x: x in ['Skipped', 'MemoryLimit']).all())

### Near-linear scaling and 500x speedup

In [ ]:
# Scaling of A*PA
df = read_results("results/scaling_n.json")
s = df.pivot(index=['errorrate','algo_key', 'r'], values='s_per_pair', columns='length')
fits = s.apply(lambda row: np.polyfit(np.log(row.index), np.log(row), 1)[0], axis=1).round(2)
#display(fits)
display(fits.loc[([0.05, 0.15], ['sh', 'gcsh-dt'])])
print('Best fit for SH at 5% for r=1 is ', fits[0.05]['sh'][1], ' and GCSH-DT at 15% for r=2 is ', fits[0.15]['gcsh-dt'][2])

# Compute max speedup over edlib and biwfa for e=5%, r=1
df = df[df.errorrate == 0.05]
df = df[df.r != 2]
s = df[df.length == 10**7].pivot(index='algo_key', values='s_per_pair', columns='errorrate')
#print('Time for n=10^7, e=5%, r=1')
#display(s)
s /= s.loc['sh-dt']
s = s.round(1)
print('Speedup of SH+DT')
display(s)
speedup=500
print('Speedup of SH+DT over edlib at 5%, 10^7 is ', s[0.05]['edlib'], f' > {speedup}')
print('Speedup of SH+DT over biwfa at 5%, 10^7 is ', s[0.05]['biwfa'], f' > {speedup}')
assert s[0.05]['edlib'] > speedup
assert s[0.05]['biwfa'] > speedup



### Appendix C: larger figures

In [ ]:
df = read_results("results/scaling_n.json")
#df = df[df.algo_key.isin(['edlib', 'biwfa', 'sh', 'gcsh-dt'])]

for (e, g) in df.groupby('errorrate'):
    cone = lambda df: (df['algo_key'] == 'sh') & (df['r'] == (1 if e <= 0.05 else 2))
    file = f'scaling_n_e{e:.2f}_large'
    plot(g, file=f'scaling_n_e{e}_large', x='length', y='s_per_pair', fit=True, cone=cone, cone_x = 3000, width=1.57*4.4, height=2.2*3, ylim = (10**-3.9, 10**3.9))
plt.show()

### Appendix C: Equivalent band

In [ ]:
df = read_results("results/scaling_n.json")
df = df[~df.algo_key.isin(['edlib', 'biwfa', 'dijkstra', 'dijkstra-dt', 'sh-noprune', 'gap', 'frequency'])]

for ((e, r), g) in df.groupby(['errorrate', 'r']):
    if e==0.15 and r==1: continue
    if e == 0.05:
        ylim = (1, 2.1)
    else:
        ylim = (1, 19)
    plot(g, file=f'band_e{e:.2f}_r{r}', x='length', y='band', connect=True, line_labels=True,
                 ylog=False, ylim=ylim,  width=4.4, height=3,
                )
plt.show()

## Scaling with divergence (Fig 4c, Appendix)

In [ ]:
df = read_results("results/scaling_e.json")
df = df[df.algo_key != 'unknown']
display(df[df.output_Err.notna()].output_Err.unique())
df = df[df.s_per_pair < 100]
plot(df, file=f'scaling_e_zoom', x='divergence', y='s_per_pair', size=None, xlog=False, ylog=False, connect=True, line_labels=True,
             ylim = (0, 0.4),
             width=4.4, height=3)
plot(df, file=f'scaling_e', x='divergence', y='s_per_pair', size=None, xlog=False, ylog=False, connect=True, line_labels=True,
             ylim = (0, 99),
             width=1.3*4.4, height=1.3*3)
plt.show()

### Variance of divergence within each dataset
Each datapoint in the plots above consists of 10 sequence pairs. The reported divergence is the average of this set. Just below each threshold, there will be some sequences in the set that have divergence above the threshold. This skews the thresholds as shown slightly.

In [ ]:
df = read_results("results/scaling_e.json")
print('Max divergence per dataset')
div = df[df.divergence > 0.09].pivot_table(index='divergence', values='stats_divergence_max')
display(div)

## Human data (Fig 5, Appendix)

In [ ]:
from pathlib import Path
df = read_results("results/real.json")
def boxplot(df, file, w):
    fig, axs = plt.subplots(1, 2, figsize=(w, 3))
    for (k, g), ax in zip(df.groupby('dataset', sort=False), axs):
        print(k)
        if file == 'real_astarpa':
            g.loc[g.algo_key == 'gcsh-dt', 'algo_pretty'] = 'A*PA'
        plot(g, x='algo_pretty', y='runtime_capped', xlog=False, ylog=True, categorical=True, ylim=(0.3, 150), ax=ax)

        ax.set_xlabel(dataset_pretty[k])
        if k == 'ont-minion-ul-500k' and file in ['real_astarpa']:
            ax.set_xlabel('ONT reads + genetic var.')
      
    if file == 'real_astarpa':
        axs[1].set_yticklabels([])
        axs[1].set_ylabel(None)
    fig.subplots_adjust(wspace=.15)
    plt.savefig(f"plots/{file}.pdf", dpi=300, bbox_inches='tight')
    plt.show()

#boxplot(df[df.algo_key.isin(['edlib', 'biwfa', 'gcsh-dt'])] , 'real_astarpa', 4.5)
boxplot(df, 'real_full', 15)

### Divergence threshold 10%: All with d<10% pass

In [ ]:
df = read_results("results/real.json")
df = df[df.algo_key == 'gcsh-dt']
for key, g in df.groupby('dataset'):
    print('dataset', key)
    print('Num failures ', len(g[g.output_Err.notna()]))
    min_fail = g[g.output_Err.notna()].divergence.min()
    max_pass = g[g.output_Err.isna()].divergence.max()
    print('Min failed divergence', min_fail)
    print('Max passed divergence', max_pass)
    print()
    assert min_fail > 0.01, "Seqs below 10% divergence should pass"

### Median speedup >3x / >1.7x on human data

In [ ]:
df = read_results("results/real.json")
df = df[df.algo_key.isin(['edlib', 'biwfa', 'gcsh-dt'])]
m = df.pivot_table(index='algo_key', columns=['dataset'], values='runtime_capped', aggfunc = np.median, sort=False)
#display(m)
m /= m.loc['gcsh-dt']
print('speedup of gcsh-dt over others')
display(m)
assert m['ont-ul-500k']['edlib'] >= 3 and m['ont-ul-500k']['biwfa'] > 3
assert m['ont-minion-ul-500k']['edlib'] >= 1.7 and m['ont-minion-ul-500k']['biwfa'] > 1.7

## A*PA Parameters (Appendix)

In [ ]:
df = read_results("results/params.json")
df['rn'] = df.apply(lambda row: f"{row['r']}-{row['length']}", axis=1)
for d, g in df.groupby('errorrate'):
    plot(g, file=f'params_e{d}', x='k', y='s_per_pair', xlog=False, ylog=True, connect=True, line_labels=True,
             hue='length', style='r', xlim=(4, 26),
             width=1.3*4.4, height=1.3*3)
plt.show()

display(df[df.output_Err.notna()].output_Err.unique())

In [ ]:
from pathlib import Path
df = read_results("results/params_real.json")
def boxplot(df, file, w):
    fig = plt.figure(constrained_layout=True, figsize=(w, 3))
    subfigs = fig.subfigures(1, 2)
    for (k, g), subfig in zip(df.groupby(['dataset'], sort=False), subfigs):
        subfig.suptitle(dataset_pretty[k], y=0)
        axs = subfig.subplots(1, 2, sharey=True)
        for (r, g), ax in zip(g.groupby(['r'], sort=False), axs): 
            plot(g, x='k', y='runtime_capped', hue='r', xlog=False, ylog=True, categorical=True, ylim=(0.3, 150), ax=ax)
            ax.set_xlabel(f'$k$, {"in" if r==2 else ""}exact matches ($r={r}$)')

    plt.savefig(f"plots/{file}.pdf", dpi=300, bbox_inches='tight')
    plt.show()

boxplot(df, 'params_real', 12)

display(df[df.output_Err.notna()].output_Err.unique())

## Memory usage (Appendix)

### Synthetic memory usage

In [ ]:
df = read_results("results/memory.json")
df.memory = (df.memory/1000000).round(0).astype('int')
df.sort_values(by='errorrate', inplace=True, kind='stable')
table = df.pivot_table(index='algo_pretty', columns = 'errorrate', values = 'memory', sort=False)
display(table)
#print(table.to_latex())
assert table.max().max() < 500, "All should use at most 500MB"

### Human data memory usage

In [ ]:
df = read_results("results/real.json")
df.memory = (df.memory/1000000)
df['capped_memory'] = df.memory.fillna(1000000)
df = df[df.algo_key.isin(['edlib', 'biwfa', 'gcsh-dt'])]
table = df.pivot_table(index='algo_pretty', columns=['dataset'], values=['capped_memory', 'memory'], aggfunc={'capped_memory': np.median, 'memory': np.max}, sort=False).round(0).astype('int')
table =table.rename({'capped_memory': 'Median', 'memory': 'Max'}, axis='columns')
table = table.swaplevel(axis=1)
table.sort_index(axis=1, level=0, inplace=True, kind='stable', ascending=False)
display(table)
#print(table.to_latex())

## Runtime profile (Appendix)

In [ ]:
df = read_results("results/timing.json")
time_labels = {'precomp': 'Precomputation', 'astar': 'A*: opening & expanding', 'h': 'A*: h() evaluation', 'pruning': 'A*: Pruning matches', 'contours': 'A*: Updating contours', 'reordering': 'A*: Reordering states', 'traceback': 'Traceback'}

for c in df.columns:
    prefix = 'output_Ok_stats_t'
    if c.startswith(prefix):
        name = c[len(prefix):]
        label = time_labels.get(name, name)
        df = df.rename({c: label}, axis='columns')

# todo: stats_expanding
df = df.pivot_table(index=['divergence', 'r'], values=time_labels.values(), sort=False)
unique_errorrates = df.index.get_level_values('divergence').unique()
unique_r_values = df.index.get_level_values('r').unique()

fig, axes = plt.subplots(nrows=len(unique_r_values), ncols=len(unique_errorrates), figsize=(6, 4))

# Create a dictionary to store handles and labels for the legend
handles_dict = {}
size_factor = 0.065

for r_idx, r_val in enumerate(unique_r_values):
    for err_idx, err_val in enumerate(unique_errorrates):
        ax = axes[r_idx, err_idx]
        if (err_val, r_val) in df.index:
            temp_df = df.loc[(err_val, r_val)]
            total = temp_df.sum()
            radius = np.sqrt(total * size_factor)
            wedges, texts, autotexts = ax.pie(temp_df, autopct="", pctdistance=0.8, radius=radius, startangle = 90, colors = sns.color_palette())
            for wedge, label in zip(wedges, temp_df.index):
                handles_dict[label] = wedge
        else:
            ax.axis('off')
        if r_idx == 1:
            ax.set_xlabel(f'd={100*err_val:.0f}%', labelpad=0)
        if err_idx == 0:
            ax.set_ylabel(f'r={r_val}', labelpad=-15, rotation=0)

# Add a single legend for all pie charts
fig.legend(handles_dict.values(), handles_dict.keys(), loc='upper right')
plt.tight_layout(pad=0, w_pad=-2.5)
plt.subplots_adjust(hspace=-0.6)
plt.savefig(f"plots/timing_synthetic.pdf", dpi=300, bbox_inches='tight') 
plt.show()

In [ ]:
df = read_results("results/timing_real.json")
df = df[df.output_Ok_stats_tastar.notna()]

for c in df.columns:
    prefix = 'output_Ok_stats_t'
    if c.startswith(prefix):
        name = c[len(prefix):]
        label = time_labels.get(name, name)
        df = df.rename({c: label}, axis='columns')


fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 4))

for k, g in df.groupby('dataset'):
    ax = axes[0 if k=='ont-ul-500k' else 1]
    df = g
    df.sort_values(by='runtime', inplace=True, kind = 'stable')
    df.plot.bar(y = time_labels.values(), stacked=True, width=.9, zorder=2, ax=ax, color = sns.color_palette())
    ax.set_ylabel('Runtime per alignment [s]', rotation=0, ha="left")
    ax.set_ylim(0, 11)
    ax.set_xlabel(dataset_pretty[k])
    if k == 'ont-minion-ul-500k':
        ax.legend().remove()
    ax.tick_params(
        axis="x",
        which="both",
        bottom=False,
    )
    ax.set_xticklabels([])
    ax.locator_params(axis='y', nbins=4)
    ax.set_facecolor("#F8F8F8")
    ax.grid(False)
    ax.grid(True, axis="y", which="major", color="w")
    ax.yaxis.set_label_coords(-0.1, 1.00)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(False)
    
plt.savefig(f"plots/timing_real.pdf", dpi=300, bbox_inches='tight')    
plt.show()